In [ ]:
### The code for downloading the pretrained model, and the SimCLR class, is based on this tutorial here: https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial17/SimCLR.html

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from tqdm.notebook import tqdm
from typing import Dict
from torch.utils.data import Dataset

# Your image dataset should live here
IMAGE_DATASET_PATH = "../../data"

# Your pretrained models should live here
MODELS_DIR = "../../models"
PRETRAINED_SIMCLR_FILENAME = "SimCLR.ckpt"
PRETRAINED_SIMCLR_PATH = os.path.join(MODELS_DIR, PRETRAINED_SIMCLR_FILENAME)

In [ ]:
### Load (or download) the pretrained model

import urllib.request
from urllib.error import HTTPError

# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial17/"
os.makedirs(MODELS_DIR, exist_ok=True)

# Check whether the pretrained model file already exists locally. If not, try downloading it
if not os.path.isfile(PRETRAINED_SIMCLR_PATH):
    file_url = base_url + PRETRAINED_SIMCLR_FILENAME
    print(f"Downloading {file_url}...")
    try:
        urllib.request.urlretrieve(file_url, PRETRAINED_SIMCLR_PATH)
    except HTTPError as e:
        print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

print(f"Already downloaded pretrained model: {file_url}")


In [ ]:
### Define SimCLR class and logic for extracting feature representations

class SimCLR(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()

        self.num_workers = os.cpu_count()
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        
        # Base ResNet18 backbone (pretrained=False, because we load custom weights later, from the SimCLR checkpoint file)
        self.convnet = torchvision.models.resnet18(pretrained=False)
        
        # This is the projection head, only needed during training. For downstream tasks it is disposed of
        # and the final linear layer output is used (Chen et al., 2020) 
        self.convnet.fc = nn.Sequential(
            nn.Linear(self.convnet.fc.in_features, 4 * hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(4 * hidden_dim, hidden_dim)
        )

        self.intermediate_layers_to_capture =[]
        self.intermediate_layer_features = {}

    def load_pretrained(self, checkpoint_path=PRETRAINED_SIMCLR_PATH):
        """
        Load pretrained SimCLR weights
        """
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.load_state_dict(checkpoint['state_dict'])
        self.to(self.device)
        self.eval()

    def set_intermediate_layers_to_capture(self, layers):
        """
        Register hooks to capture features from intermediate layers
        """
        self.intermediate_layers_to_capture = layers
        intermediate_layer_features = {}

        def get_hook(layer_name):
            def hook(module, input, output):
                intermediate_layer_features[layer_name] = output.detach()
            return hook

        for layer_name in layers:
            layer = dict([*self.convnet.named_modules()])[layer_name]
            layer.register_forward_hook(get_hook(layer_name))

        self.intermediate_layer_features = intermediate_layer_features

    @torch.no_grad()
    def extract_features(self, dataset: Dataset) -> Dict[str, torch.Tensor]:
        """
        Run the pretrained SimCLR model on the image data, and capture features from final layer and intermediate layers.

        Args:
            dataset (Dataset): A PyTorch dataset containing input images and labels.

        Returns:
            Dict[str, torch.Tensor]: A dictionary containing:
                - Intermediate layer features as tensors.
                - Final layer features under 'final_layer'.
                - Labels under 'labels'.
        """
        self.convnet.fc = nn.Identity()  # Removing projection head g(.)
        self.eval()
        self.to(self.device)
        
        # Encode all images
        data_loader = DataLoader(dataset, batch_size=64, num_workers=self.num_workers, shuffle=False, drop_last=False)
        feats, labels, intermediate_features = [], [], {layer: [] for layer in self.intermediate_layers_to_capture}

        for batch_imgs, batch_labels in tqdm(data_loader):
            batch_imgs = batch_imgs.to(self.device)
            batch_feats = self.convnet(batch_imgs)
            
            feats.append(batch_feats.detach().cpu())
            labels.append(batch_labels)
            
            # Collect intermediate layer outputs
            for layer in self.intermediate_layers_to_capture:
                # Final linear layer outputs a 2d tensor; but intermediate layers don't, so we flatten them (ready for PCA etc.)
                layer_output_flattened = self.intermediate_layer_features[layer].view(self.intermediate_layer_features[layer].size(0), -1) 
                intermediate_features[layer].append(layer_output_flattened.cpu())
        
        # Concatenate results for each layer
        feats = torch.cat(feats, dim=0)
        labels = torch.cat(labels, dim=0)
        intermediate_features = {layer: torch.cat(intermediate_features[layer], dim=0) for layer in self.intermediate_layers_to_capture}

        return {**intermediate_features, 'final_layer': feats, 'labels': labels}

In [ ]:
### Prepare your image data
from torch.utils.data import TensorDataset

# Generate a random set of images (TODO: replace with your images)
num_images = 600
image_height, image_width = 224, 224
random_grayscale_images = torch.rand((num_images, 1, image_height, image_width), dtype=torch.float32)
random_rgb_images = random_grayscale_images.repeat(1, 3, 1, 1)  # Shape: (N, 3, H, W)
random_labels = torch.arange(0, num_images)
image_dataset = TensorDataset(random_rgb_images, random_labels)


In [ ]:
### Run your image data through the pretrained SimCLR model, and extract feature representations

intermediate_layers = ['layer1', 'layer2', 'layer3', 'layer4']

sim_clr = SimCLR()
sim_clr.load_pretrained()
sim_clr.set_intermediate_layers_to_capture(intermediate_layers)
feats = sim_clr.extract_features(image_dataset)

layer1_feats = feats['layer1']
layer2_feats = feats['layer2']
layer3_feats = feats['layer3']
layer4_feats = feats['layer4']
final_layer_feats = feats['final_layer']